In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [3]:
load_dotenv()

True

#### **Aşağıda ismimizi verdik ve bir cevap döndü. ikinci mesaj olarak ismimizin ne oldupunu sorsaydık bunu bilmediğini söyleyecekti. çünkü her yeni mesajda yeni bir sesssion açıyormuşuz gibi davranıyor.**

In [5]:
model = ChatOpenAI(model="gpt-3.5-turbo")

message = HumanMessage(content="Hello, my name is Yusuf")
response = model.invoke([message])
response.content

'Hello Yusuf, nice to meet you. How can I assist you today?'

#### **Burada bir session açıp önce kendi mesajımızı, sonra AI ın verdiği cevabı yazdırdık. sonrasında ismimizi sorduğumuzda buna cevap verdi, konuşma bir session içindeymiş gibi davrandı. Fakat bu yöntem mantıksız ve işlevsel değil. Elimizle session açıp mesaj girerek olmaz**

In [6]:
messages = [
    HumanMessage(content="Hello, my name is yusuf"),
    AIMessage(content="Hello Yusuf, how can i help you today?"),
    HumanMessage(content="What is my name?")
]

response = model.invoke(messages)
response.content

'Your name is Yusuf.'

In [5]:
model = ChatOpenAI(model="gpt-3.5-turbo")

store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model
config = {"configurable": {"session_id": "abcde123"}}
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

while True: 
    user_input = input(">")

    if user_input.lower() in ["exit", "quit", "çıkış"]:
        print("Session sonlandiriliyor.")
        break

    response = with_message_history.invoke(
        [
            HumanMessage(content=user_input)
        ],
        config=config,
    )
    print(response.content)

Hello Yusuf! How can I assist you today?
Your name is Yusuf.
Döngü sonlandırılıyor.


#### **Çıktıyı stream olarak alma:**

In [7]:
model = ChatOpenAI(model="gpt-3.5-turbo")

store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model
config = {"configurable": {"session_id": "abcde123"}}
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

while True: 
    user_input = input(">")

    if user_input.lower() in ["exit", "quit", "çıkış"]:
        print("Session sonlandiriliyor.")
        break

    for r in with_message_history.stream(
        [
            HumanMessage(content=user_input)
        ],
        config=config,
    ):
        print(r.content, end=" ")

 Sure !  Gravity  is  the  force  that  pulls  objects  with  mass  towards  each  other .  It  is  what  keeps  us  grounded  on  Earth  and  is  responsible  for  the  motion  of  planets ,  stars ,  and  galaxies .  According  to  Albert  Einstein 's  theory  of  general  rel ativity ,  gravity  is  the  result  of  the  curvature  of  spac etime  caused  by  massive  objects .  Session sonlandiriliyor.
